In [37]:
import os
import pandas as pd
from dbfread import DBF

In [38]:
dfAnalysisSets = pd.read_csv('output\\_analysis_sets.csv')
dfAnalysisSets

,date_range_name,data_ranges,mode_name,mode_description,zone_set_name,analysis_set_name
0,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",TRU,Truck,StreetLight_TAZ_RecreationZones_CacheCo,Recreation_Zones_CacheCo_2019_01_TRU
1,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",BIC,Bicycle,StreetLight_TAZ_RecreationZones_CacheCo,Recreation_Zones_CacheCo_2019_01_BIC
2,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",PED,Pedestrian,StreetLight_TAZ_RecreationZones_CacheCo,Recreation_Zones_CacheCo_2019_01_PED
3,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",ALL,All_Vehicles,StreetLight_TAZ_RecreationZones_CacheCo,Recreation_Zones_CacheCo_2019_01_ALL
4,2019_02,"[{'start_date': '02/01/2019', 'end_date': '02/...",TRU,Truck,StreetLight_TAZ_RecreationZones_CacheCo,Recreation_Zones_CacheCo_2019_02_TRU
...,...,...,...,...,...,...
1371,2022_03,"[{'start_date': '03/01/2022', 'end_date': '03/...",ALL,All_Vehicles,Summit Wasatch Rec,Recreation_Zones_Summit-Wasatch_2022_03_ALL
1372,2022_04,"[{'start_date': '04/01/2022', 'end_date': '04/...",TRU,Truck,Summit Wasatch Rec,Recreation_Zones_Summit-Wasatch_2022_04_TRU
1373,2022_04,"[{'start_date': '04/01/2022', 'end_date': '04/...",BIC,Bicycle,Summit Wasatch Rec,Recreation_Zones_Summit-Wasatch_2022_04_BIC
1374,2022_04,"[{'start_date': '04/01/2022', 'end_date': '04/...",PED,Pedestrian,Summit Wasatch Rec,Recreation_Zones_Summit-Wasatch_2022_04_PED


In [62]:
# read DBFs to get zones
zoneSetDir = 'E:\\GitHub\\StreetLight-Recreation-and-Transit\\Zone-Sets\\redownloaded-from-streetlight\\'
analysisSetDir = "E:\\GitHub\\StreetLight-Recreation-and-Transit\output\\"
indivZonesDatDir = "E:\\GitHub\\StreetLight-Recreation-and-Transit\output\\results-by-indiv-zones\\"

dfZoneSets = dfAnalysisSets[['zone_set_name']].drop_duplicates()

dfZoneSets

for indexZS, rowZS in dfZoneSets.iterrows():
    zsName = rowZS['zone_set_name'].replace(' ','-')
    dbfName = (zoneSetDir + "UDOT---PPU_" + zsName + '_ZoneSet\\zone_set_' + zsName.replace('-','_') + '.dbf').replace('_Moab.dbf','.dbf')
    print(zsName)

    _dfZones = pd.DataFrame(DBF(dbfName, load=True))
    _dfZones['name'] = _dfZones['name'].astype(str)

    #display(_dfZoneSet)
    dfAnalysisSets_ZoneSet = dfAnalysisSets[dfAnalysisSets['zone_set_name']==rowZS['zone_set_name']]

    _dfZoneSetResults = pd.DataFrame()

    # read in data for each zone set
    for indexA, rowA in dfAnalysisSets_ZoneSet.iterrows():
        #print(rowA['analysis_set_name'])

        _df = pd.read_csv(os.path.join(analysisSetDir,rowA['analysis_set_name']+'.csv'),low_memory=False)

        _df['mode_name'] = rowA['mode_name']
        _df['date_range_name'] = rowA['date_range_name']

        if(rowA['mode_name']!='TRU'):
            _df['truck_weight'] = 'na'

        _dfZoneSetResults = pd.concat([_dfZoneSetResults,_df])

    col = _dfZoneSetResults.pop('truck_weight')  # remove column 'B' from the dataframe and store it in 'col'
    _dfZoneSetResults.insert(len(_dfZoneSetResults.columns), 'truck_weight', col)  # insert column 'B' at the end of the dataframe
    
    _dfZoneSetResults['OName'] = _dfZoneSetResults['OName'].astype(str)
    _dfZoneSetResults['DName'] = _dfZoneSetResults['DName'].astype(str)
    #display(_dfZoneSetResults)

    # write out files for each zone
    for indexZ, rowZ in _dfZones.iterrows():
        print(rowZ['name'])

        _dfZoneDataAsO = _dfZoneSetResults[_dfZoneSetResults['OName']==rowZ['name']]
        _dfZoneDataAsD = _dfZoneSetResults[_dfZoneSetResults['DName']==rowZ['name']]

        _dfZoneDataAsO.to_csv(indivZonesDatDir + rowZS['zone_set_name'] + '__' + rowZ['name'] + '_asOrig.csv',index=False)
        _dfZoneDataAsD.to_csv(indivZonesDatDir + rowZS['zone_set_name'] + '__' + rowZ['name'] + '_asDest.csv',index=False)

StreetLight_TAZ_RecreationZones_CacheCo
50880_2
50888_2
StreetLight_TAZ_RecreationZones_UDOT
1052_2
1055_1
15267_1
15267_2
15267_3
17145_2
19073_2
19073_3
19073_4
21345_0
21350_0
21359_1
21359_2
21359_3
37023_2
37023_3
37023_4
37163_1
37163_2
47133_1
47133_2
47133_3
47133_4
51016_1
51016_2
51016_3
51250_1
51250_2
55091_1
55091_2
57058_0
57072_1
57072_2
9003_1
9011_1
StreetLight_TAZ_RecreationZones_WashingtonCo
530008_1
530458_2
530591_1
530597_1
530597_2
530695_1
530695_2
530695_3
530695_4
530699_1
530699_2
530701_1
530701_2
530701_3
530701_4
530702_1
530702_2
530788_1
530804_1
530810_1
530818_1
530823_1
530823_2
530823_3
530823_4
530823_5
StreetLight_TAZ_RecreationZones_WFRC
110118_2
110134_2
110164_2
110189_2
110198_2
110224_2
110248_2
110314_2
350068_2
350068_3
350069_2
350069_3
350078_2
350197_2
350197_3
350197_5
350264_2
350599_2
350626_2
350790_2
351002_2
351002_3
351012_2
351031_2
351211_2
351232_2
351232_3
351243_2
351243_3
351243_4
351243_5
351247_2
351250_2
351251_2
351251_3
